# Aula 17: ACID

# Notebook B

### Ponto 1

In [ ]:
import mysql.connector
import os
from functools import partial
from dotenv import load_dotenv

load_dotenv(override=True)

def get_connection_helper(database):

    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            print("Executando query:")
            cursor.execute(query, args)
            for result in cursor:
                print(result)

    connection = mysql.connector.connect(
        host=os.getenv('MD_DB_SERVER'),
        user=os.getenv('MD_DB_USERNAME'),
        password=os.getenv('MD_SENHA_BANCO'),#'MD_DB_PASSWORD'),
        database=database,
    )
    return connection, partial(run_db_query, connection)


connection2, db2 = get_connection_helper("tranqueira")

<img src="pare.png" width=150px/>

Volte para o notebook A.

---

## `SERIALIZABLE`

### Ponto 2

Vamos mudar o nível de isolamento aqui também para `SERIALIZABLE`:

In [ ]:
db2('SET SESSION TRANSACTION ISOLATION LEVEL SERIALIZABLE')
db2('START TRANSACTION')

<img src="pare.png" width=150px/>

Volte para o notebook A.

---

### Ponto 3

Rode esta célula. Você vai ver que o notebook vai ficar parado aqui, esperando o resultado da query. Se você esperar bastante tempo (1 ou 2 minutos) você verá a seguinte mensagem:

```DatabaseError: 1205 (HY000): Lock wait timeout exceeded; try restarting transaction```

In [ ]:
db2('INSERT INTO perigo(Nome) VALUES ("Astral")')
db2('SELECT * FROM perigo')

O que aconteceu?

- O notebook A adquiriu a trava (lock) da tabela `perigo`;
- O notebook B tentou adquirir essa trava para executar a query;
- Para evitar deadlock o MySQL implementou um mecanismo de timeout na espera pela aquisição da trava.

Agora rode de novo esta célula, mas volte rapidamente para o notebook A e siga as instruções.

<img src="pare.png" width=150px/>

---

Observe que ao fechar a transação em A, liberamos a trava da tabela `perigo` e permitimos que a transação de B continue a ocorrer.

Aqui, o que aconteceria se tentássemos obter apenas a primeira linha (`WHERE id=1`)? Haveria **lock** mesmo assim?

<div class="alert alert-success">

Sua resposta AQUI!

</div>

Não prossiga. Volte para o notebook A e continue a atividade. Você retornará a este ponto aqui em breve.

<img src="pare.png" width=150px/>

---

Vamos encerrar a transação com o cancelamento da operação de inserção:

In [ ]:
db2('ROLLBACK')

<img src="pare.png" width=150px/>

Volte para o notebook A.

## `REPEATABLE READ`

### Ponto 4

Vamos mudar o nível de isolamento aqui também:

In [ ]:
db2('SET SESSION TRANSACTION ISOLATION LEVEL REPEATABLE READ;')

<img src="pare.png" width=150px/>

Volte para o notebook A.

### Ponto 5

In [ ]:
db2('START TRANSACTION')

<img src="pare.png" width=150px/>

Volte para o notebook A.

---

### Ponto 6

Rode a célula abaixo:

In [ ]:
db2('INSERT INTO perigo(Nome) VALUES ("Ortopédico")')
db2('SELECT * FROM perigo')

Observe que, diferente da situação anterior, não houve travamento! 

<img src="pare.png" width=150px/>

Volte para o notebook A.

---

### Ponto 7

Agora confirme a transação:

In [ ]:
db2('COMMIT')

Observe que a mudança da tabela foi registrada:

In [ ]:
db2('SELECT * FROM perigo')

<img src="pare.png" width=150px/>

Volte para o notebook A.

### Ponto 8

Vamos abrir uma nova transação e tentar mudar o nome 'Mental' para 'Psicológico':

In [ ]:
db2('START TRANSACTION')

In [ ]:
db2('UPDATE perigo SET Nome = "Psicológico" WHERE Nome = "Mental"')

In [ ]:
db2('SELECT * FROM perigo')

<img src="pare.png" width=150px/>

Volte para o notebook A.

In [ ]:
db2('COMMIT')

### Ponto 9

Feche a conexão e volte para o notebook A.

In [ ]:
connection2.close()